In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install us


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import requests
import json
import us

def retrieve_census_data(api_key, state, county):
    # Define the API key and URL
    API_URL = 'https://api.census.gov/data/2019/acs/acs5'

    FIELDS = ['B25077_001E', 'C24050_001E', 'C24050_011E', 'C24050_014E', 'B25013_009E', \
              'B25013_008E', 'B25013_007E', 'B25002_001E', 'B25002_002E', 'B01003_001E', \
              'B25031_001E', 'B25103_001E', 'B25118_014E', 'B25118_015E', 'B25118_016E', \
              'B25118_017E', 'B25118_018E', 'B25118_019E', 'B25118_020E', 'B25118_021E', \
              'B25118_022E', 'B25118_023E', 'B25118_024E', 'B25118_025E', 'B25118_001E', \
              'B25026_010E', 'B25026_011E', 'B25026_012E', 'B25026_013E', 'B25026_014E', \
              'B25026_015E', 'B25046_003E', 'B08013_001E', 'B25003I_003E', 'B25003A_003E', \
              'B25003B_003E', 'B25003C_003E', 'B25003D_003E', 'B25003E_003E', \
              'B25003F_003E', 'B25003G_003E', 'B25003H_003E', 'B19013_001E', 'B25119_003E', \
              'B25010_002E', 'B25010_003E', 'B25009_011E', 'B25009_012E', 'B25009_013E', \
              'B25009_014E', 'B25009_015E', 'B25009_016E', 'B25009_017E', \
              'B25032_013E', 'B25032_014E', 'B25032_015E', 'B25032_016E', 'B25032_017E', \
              'B25032_018E', 'B19057_001E', 'B19057_002E', 'B19057_003E', 'B25042_001E', 'B25042_003E', \
              'B25042_004E', 'B25042_005E', 'B25042_006E', 'B25042_007E', 'B25042_008E', \
              'B25042_010E', 'B25042_011E', 'B25042_012E', 'B25042_013E', 'B25042_014E', 'B25042_015E']

    # Break the list of fields into chunks of 50
    chunks = [FIELDS[i:i+50] for i in range(0, len(FIELDS), 50)]

    # Initialize an empty DataFrame
    data = pd.DataFrame()

    for chunk in chunks:
        params = {
            'get': ','.join(chunk),
            'for': 'block group:*',
            'in': f'state:{state} county:{county}',
            'key': api_key
        }

        # Request the data from the API
        response = requests.get(API_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            json_data = json.loads(response.text)
            chunk_data = pd.DataFrame(json_data[1:], columns=json_data[0])

            # Convert numeric columns to appropriate dtypes
            for column in chunk_data.columns:
                chunk_data[column] = pd.to_numeric(chunk_data[column], errors='ignore')

            # If this is the first chunk, initialize the data DataFrame
            if data.empty:
                data = chunk_data
            else:
                # If this is not the first chunk, merge it with the existing data
                data = pd.merge(data, chunk_data, on=['state', 'county', 'tract', 'block group'])
        else:
            print(f'Request failed with status code {response.status_code}. Message: {response.text}')

    data['country'] = 'United States'

    # Now that all chunks have been processed, perform some final cleaning
    data['state_name'] = data['state'].apply(lambda x: us.states.lookup(str(int(x))) if pd.notnull(x) else x)

    # Display the first few rows of the data
    return data.head()

# Prompt the user to enter the desired state and county codes
state_code = input("Enter the state code (e.g., 30 for Montana & 06 for california): ")
county_code = input("Enter the county code (e.g., 111 for the specific county & 037 respectively): ")

# Retrieve the Census data for the specified state and county
census_data = retrieve_census_data('faad7551acd1bf70615d1197d3fac92869820e1a', state_code, county_code)

# Display the retrieved data
census_data


,B25077_001E,C24050_001E,C24050_011E,C24050_014E,B25013_009E,B25013_008E,B25013_007E,B25002_001E,B25002_002E,B01003_001E,...,B25042_007E,B25042_008E,B25042_010E,B25042_011E,B25042_012E,B25042_013E,B25042_014E,B25042_015E,country,state_name
0,589700,NaN,NaN,NaN,NaN,NaN,NaN,168,168,414,...,48,23,0,0,0,0,0,0,United States,None
1,496900,NaN,NaN,NaN,NaN,NaN,NaN,774,736,2242,...,39,9,36,73,78,35,0,11,United States,None
2,277100,NaN,NaN,NaN,NaN,NaN,NaN,383,346,1847,...,0,0,0,19,20,108,23,0,United States,None
3,2000001,NaN,NaN,NaN,NaN,NaN,NaN,371,287,736,...,32,100,17,11,12,22,0,0,United States,None
4,593800,NaN,NaN,NaN,NaN,NaN,NaN,320,249,1050,...,43,0,0,0,8,108,0,0,United States,None
